# View SL , View spectra one by one

- **Part 1**
- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- creation date : 2022/12/23
- last update : 2023/03/20
- update : 2023/09/27 @CC kernel conda_prospector_py310 
- FORS2 : https://github.com/ericnuss/fors2
- Result of StarLight : /sps/lsst/groups/photoz/fors2/FORS2/ResStarlight



          /Users/dagoret/MacOSX/GitHub/LSST/FORS2>ls -l
              total 0
              drwxrwsr-x  3 dagoret  staff   96 Aug 21  2018 ResStarlight
              drwxr-xr-x  9 dagoret  staff  288 Dec 19 18:01 fors2

## Import

In [ ]:
import os,sys,re
from astropy.io import fits
from astropy.table import Table

In [ ]:
# Import some generally useful packages

import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

from matplotlib.backends.backend_pdf import PdfPages

%matplotlib inline
import pandas as pd
from itertools import cycle, islice


In [ ]:
from scipy import ndimage

#from scipy import interp as scinterp
from scipy.interpolate import interp1d

import glob
import collections
#from def_raw_seds import *

In [ ]:
# to enlarge the sizes
params = {'legend.fontsize': 'large',
          'figure.figsize': (8, 4),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
plt.rcParams.update(params)

In [ ]:
import matplotlib.offsetbox
props = dict(boxstyle='round',edgecolor="w",facecolor="w", alpha=0.5)

In [ ]:
from def_raw_seds import *

In [ ]:
from raw_data_analysis import *

In [ ]:
from spectroscopy import *

In [ ]:
# Import the primary photometry classes from rubin_sim.photUtils

#import rubin_sim.phot_Utils.Bandpass as Bandpass
#import rubin_sim.phot_Utils.Sed as Sed
#from rubin_sim.data import get_data_dir

In [ ]:
#pysynphot_root_path=os.environ['PYSYN_CDBS']
#path_sed_calspec=os.path.join(pysynphot_root_path,'calspec')
# pysynphot
#import pysynphot as S
#S.refs.setref(area=1)
#S.refs.getref()

## Configuration

### SL

## Start

### Catalog

In [ ]:
t = Table.read(filename_fits_catalog)

In [ ]:
t

### List

In [ ]:
lst=fors2_jct_SL() #remet les spectres dans le rest frame (ie a z=0)

In [ ]:
lst = sorted(lst)
print(lst)

In [ ]:
all_specfilenames = os.listdir(path_BC)

In [ ]:
all_specfilenames = sorted(all_specfilenames)

In [ ]:
all_numbers = [ int(re.findall('^SPEC(.*)_HZ4_BC[.]txt$',fn)[0]) for fn in all_specfilenames ]

In [ ]:
all_numbers = sorted(all_numbers)

In [ ]:
NSED = len(all_numbers)
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=NSED)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
cut = t["ID"] == 3
t[cut]

In [ ]:
all_spectra = []
all_wl = []
all_tag = []
all_sed = []

for the_num in all_numbers:
    tcut = t["ID"] == the_num
    redshift = t[tcut]["z"].value[0]
    lines = t[tcut]["Lines"].value[0]
    filename = f"SPEC{the_num}_HZ4_BC.txt"
    tag = f"SPEC{the_num} , z={redshift:.2f}, lines={lines}"
    fullfilename = os.path.join(path_BC,filename)
    the_sed=SED(fullfilename,label=tag)
    all_sed.append(the_sed)
    arr = np.loadtxt(fullfilename)
    all_wl.append(arr[:,0])
    all_spectra.append(arr[:,1])
    all_tag.append(tag)
 

In [ ]:
#plt.figure(figsize=(16,20))
for idx in range(NSED):
    plt.figure(figsize=(16,2))
    plt.plot(all_wl[idx],all_spectra[idx], color=all_colors[idx],label=all_tag[idx])
    plt.legend(loc="upper right")
    plt.grid()
    plt.axvline(4150,color="k")
    plt.axvline(4250,color="k")
    plt.ylim(-0.02,0.4)
    plt.xlim(0.,10000.)
    plt.show()


In [ ]:
for index, the_sed in enumerate(all_sed):
    plt.figure(figsize=(16,2))
    
    the_label=the_sed.label
    
        
    plt.plot(the_sed.wave,the_sed.flux, color=all_colors[index],label=the_label)
    plt.legend(loc="upper right")
    plt.grid()
    plt.axvline(4150,color="k")
    plt.axvline(4250,color="k")
    #plt.ylim(-0.02,0.4)
    plt.xlim(0.,10000.)
    plt.show()